In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
data=tfds.load(name="malaria",as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteGV5ASK/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
model=tf.keras.Sequential(
    [

        tf.keras.layers.Input(shape=(224,224,3)),
        tf.keras.layers.Conv2D(8,3,activation="relu"),
        tf.keras.layers.MaxPool2D(2),
        tf.keras.layers.Conv2D(16,3,activation="relu"),
        tf.keras.layers.MaxPool2D(2,strides=2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32,activation="relu"),
        tf.keras.layers.Dense(16,activation="relu"),
        tf.keras.layers.Dense(2,activation="softmax"),
    ]
)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 8)       224       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 111, 111, 8)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 16)      1168      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 46656)             0         
                                                                 
 dense_6 (Dense)             (None, 32)               

In [ ]:
def rescaling_resizing(img,lab):
  return tf.image.resize(img,(224,224))/255.,lab
data=data["train"].map(rescaling_resizing).batch(16)


In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
model.fit(data.take(100),epochs=2)

Epoch 1/2
100/100 [==============================] - 47s 460ms/step - loss: 0.6865 - accuracy: 0.5781
Epoch 2/2
100/100 [==============================] - 48s 480ms/step - loss: 0.6316 - accuracy: 0.6381


In [ ]:
def representive_data_gen():
  for d,l in data.take(20):
    yield [d]

<generator object representive_data_gen at 0x7f06c91eeb20>

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.inference_input_type=tf.uint8
converter.inference_output_type=tf.uint8
converter.representative_dataset=representive_data_gen

In [ ]:
tf_lite_model=converter.convert()

In [ ]:
import pickle
with open("tf_lite.pkl","wb") as f:
  pickle.dump(tf_lite_model,f)

In [ ]:
model.save("tf_original.h5")

In [ ]:
!pip install --extra-index-url https://www.google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://www.google-coral.github.io/py-repo/
Could not fetch URL https://www.google-coral.github.io/py-repo/tflite-runtime/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='www.google-coral.github.io', port=443): Max retries exceeded with url: /py-repo/tflite-runtime/ (Caused by SSLError(CertificateError("hostname 'www.google-coral.github.io' doesn't match either of '*.github.io', 'github.io', '*.github.com', 'github.com', 'www.github.com', '*.githubusercontent.com', 'githubusercontent.com'"))) - skipping
ERROR: Could not find a version that satisfies the requirement tflite_runtime (from versions: none)
ERROR: No matching distribution found for tflite_runtime


In [ ]:
with open("tf_lite.pkl","rb") as f:
  mo=pickle.load(f)

In [ ]:
!python --version

Python 3.10.12


In [ ]:
# !pip install fastapi

In [ ]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.365s
